In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import nivapy3 as nivapy

# Connect to db
engine = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


# Critical Loads Workflow: data for 1983 to 1992

There is a gap in the Critical Loads processing between 1983 and 1992. To fill this, Kari has received two datasets from Wenche Aas, the first covering 1983 to 1987 and the second 1988 to 1992. See e-mail received 18.03.2019 at 16.15 for details.

The data supplied by NILU use the old BLR grid and therefore the work described here matches the "new NIVA method; old deposition grid" workflow described in [this notebook](https://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow.ipynb). Note that Kari is only interested in the calculations for **water** and **vegetation**; exceedances for soil have been omitted.

## 1. Vegetation

### 1.1. Upload new data to database

In [2]:
# ID for new series
ser_id = 30

# Create new series for this data period
df = pd.DataFrame({'dep_series_id':ser_id,
                   'name':'Middel 1988-1992',
                   'description':'Fordelt til BLR av NILU 2019 (Wenche Aas; old method)'},
                  index=[0,])

## Add to db
#df.to_sql('dep_series_definitions', con=engine, schema='resa2', 
#          if_exists='append', index=False)

In [3]:
# Read data
in_dat = r'../../../processing_1983-1992/dep_data/dep_niva_8892.dat'
df = pd.read_csv(in_dat, delim_whitespace=True, index_col=0)
df.index.name = 'blr'
del df['lat'], df['long']

# Map headings to db par IDs
par_dict = {'atnsss8892':4,  # Non-marine S
            'atn8892':8}     # N (total)

# Rename cols
df.rename(par_dict, inplace=True, axis=1)
df.reset_index(inplace=True)

# Melt 
df = pd.melt(df, var_name='parameter_id', id_vars='blr')

# Add series ID
df['dep_series_id'] = ser_id

## Add to db
#df.to_sql('dep_blr_values', con=engine, schema='resa2', 
#          if_exists='append', index=False)

### 1.2. Read lookup table link vegetation classes to critical loads

In [4]:
# Read lookup table
in_xlsx = r'../../../vegetation/sat_veg_land_use_classes.xlsx'
df = pd.read_excel(in_xlsx, sheet_name='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

,CL_100smgN/m2/yr
NORUTcode,
1,5
2,5
3,5
4,10
5,10


### 1.3. Reclassify

In [ ]:
## Input national veg map
#in_tif = r'../../../gis/raster/sat_veg_30m_all.tif'
#
## Output geotiff for critical loads values
#out_tif = r'../../../gis/raster/sat_veg_30m_cr_lds_div100.tif'
#
## Mask raster for land defined by BLR
#mask_tif = r'../../../gis/raster/blr_land_mask.tif'
#
## Reclassify
#cl.reclassify_raster(in_tif, mask_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [5]:
# Get all dep values
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id = 8 "
       "AND dep_series_id IN (29, 30)")

df = pd.read_sql(sql, engine)

# Sum N_oks and N_red
df = df.groupby(['blr', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
names_dict = {29:'Ndep8387',   
              30:'Ndep8892'}
df.rename(names_dict, inplace=True, axis=1) 

print (np.nanmin(df.values), np.nanmax(df.values))

df.head()

64 2678


,Ndep8387,Ndep8892
blr,,
58006001,1721,1844
58006002,1800,2324
58006003,1787,2168
58006004,1739,2041
58006005,1800,2085


### 1.5. Loop over datasets

In [6]:
%%time

# Path to raw BLR shapefile
in_shp = r'../../../gis/shapefiles/blrgrid_uten_grums_utm_z33n.shp'

# Snap raster
snap_tif = r'../../../gis/raster/sat_veg_30m_all.tif'

# Critical loads raster
cl_tif = r'../../../gis/raster/sat_veg_30m_cr_lds_div100.tif'

# Container for output
data_dict = {'series':[],
             'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in df.columns:    
    
    print ('Processing: %s' % ser)
    print ('    Building deposition shapefile...')
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['BLR', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='BLR')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = r'../../../gis/shapefiles/dep_%s.shp' % ser
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print ('    Rasterising shapefile...')
    
    # Output BLR raster
    dep_tif = r'../../../gis/raster/dep_%s_30m.tif' % ser

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print ('    Calculating exceedance...')
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['series'].append(ser)
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print ('    Saving exceedance grid...')
    ex_tif = (r'../../../gis/raster/exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    
    print ('    Done.')

# Build output df
ex_df = pd.DataFrame(data_dict)
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df.index = ex_df['series']
del ex_df['series']
ex_df = ex_df.round(0).astype(int)

# Save
out_csv = r'../../../vegetation/nor_prop_exceed_88-92.csv'
ex_df.to_csv(out_csv)

print ('Finished.')

Processing: Ndep8387
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...


C:\Data\Anaconda3\envs\dstoolkit\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in greater
C:\Data\Anaconda3\envs\dstoolkit\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in less


    Saving exceedance grid...
    Done.
Processing: Ndep8892
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Finished.
Wall time: 4min 23s


### 1.6. Print national exceedance summary for vegetation

In [7]:
# Exceedance summary
ex_df

,nor_area_km2,ex_area_km2,ex_pct
series,,,
Ndep8387,320584,108517,34
Ndep8892,320584,86356,27


## 2. Water

### 2.1. Extract deposition data

**Note:** Remember to add the new `dep_series_id` to the query below.

In [8]:
# Get dep data
sql = ("SELECT a.blr, b.name AS series, c.name AS par, a.value AS dep "
       "FROM resa2.dep_blr_values a, "
       "resa2.dep_series_definitions b, "
       "resa2.air_parameter_definitions c "
       "WHERE a.parameter_id IN (8, 4) "
       "AND a.dep_series_id IN (29, 30) "
       "AND a.parameter_id = c.parameter_id "
       "AND a.dep_series_id = b.dep_series_id")

dep_df = pd.read_sql(sql, engine)

# Tidy
dep_df['series'] = dep_df['series'].str[7:]
dep_df['par'] = dep_df['par'].str[:1]

# Group N(oks) and N(red)
dep_df = dep_df.groupby(['blr', 'series', 'par']).sum()

# Unstack
dep_df = dep_df.unstack('par')
dep_df.columns = dep_df.columns.get_level_values('par')

# Convert to meq/l
dep_df['N'] = dep_df['N'] / 14.
dep_df['S'] = dep_df['S']*2. / 32.06

# Unstack
dep_df = dep_df.unstack('series')

# Flatten col index
dep_df.columns = ['%s_%s' % (p, d) for p, d in zip(dep_df.columns.get_level_values('par'), 
                                                   dep_df.columns.get_level_values('series'))]

dep_df.head()

,N_1983-1987,N_1988-1992,S_1983-1987,S_1988-1992
blr,,,,
58006001,122.928571,131.714286,75.857767,80.286962
58006002,128.571429,166.000000,81.597006,101.122895
58006003,127.642857,154.857143,82.470368,94.759825
58006004,124.214286,145.785714,82.158453,88.521522
58006005,128.571429,148.928571,80.286962,90.642545


### 2.2. Get critical loads

In [9]:
# Get dep data
sql = ("SELECT blr, claoaavaroaa, eno3fl, clminn, clmaxnoaa, clmaxsoaa "
       "FROM resa2.cla")

cl_df = pd.read_sql(sql, engine)

# Set index
cl_df.index = cl_df['blr']
del cl_df['blr']

# Add CLSmin as 0
cl_df['clmins'] = 0

cl_df.head()

,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,
60506013,46.689055,20.499,3.204124,59.153507,47.083568,0
60508003,47.240467,8.988,3.204124,70.544508,48.610725,0
60509002,48.396218,0.029,14.635616,100.955383,51.368535,0
60511002,11.156834,0.080,7.077605,27.656760,11.903564,0
60511006,34.764804,1.081,42.316831,121.992502,38.498971,0


### 2.3. Calculate exceedances

#### 2.3.1. Check data and join

In [10]:
# Check dfs are compatible
assert len(dep_df) == len(cl_df), 'Lengths of dataframes do not match.'
assert dep_df.index.is_unique, 'dep_df has duplicated BLR IDs.'
assert cl_df.index.is_unique, 'cl_df has duplicated BLR IDs.'

# Join
df = dep_df.join(cl_df, how='left')

# Fill NaN with 0
df.fillna(0, inplace=True)

df.head()

,N_1983-1987,N_1988-1992,S_1983-1987,S_1988-1992,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
blr,,,,,,,,,,
58006001,122.928571,131.714286,75.857767,80.286962,28.533538,35.544,3.204124,40.514435,29.056767,0
58006002,128.571429,166.000000,81.597006,101.122895,32.029772,27.861,3.204124,43.473268,32.471152,0
58006003,127.642857,154.857143,82.470368,94.759825,20.888186,25.137,3.204124,29.564269,21.184960,0
58006004,124.214286,145.785714,82.158453,88.521522,18.914393,36.846,25.972633,49.911208,19.189395,0
58006005,128.571429,148.928571,80.286962,90.642545,20.463044,47.587,3.204124,30.057701,20.846989,0


#### 2.3.2. SSWC model

**Note:** Remember to update the `periods` list below.

In [11]:
# Define time periods
periods = ['1983-1987', '1988-1992']

# Copy df
sswc_df = df.copy()

# Loop over periods
for per in periods:
    sswc_df['sswc_ex_%s' % per] = (sswc_df['S_%s' % per] + 
                                   sswc_df['eno3fl'] - 
                                   sswc_df['claoaavaroaa'])

# Get cols of interest
cols = ['sswc_ex_%s' % per for per in periods]
sswc_df = sswc_df[cols]

# Set values below 0 to 0
sswc_df[sswc_df<0] = 0

# Write to output
out_path = r'../../../water/sswc_exceedance_83-92.csv'
sswc_df.to_csv(out_path)

sswc_df.head()

,sswc_ex_1983-1987,sswc_ex_1988-1992
blr,,
58006001,82.868229,87.297424
58006002,77.428234,96.954123
58006003,86.719182,99.008639
58006004,100.090060,106.453129
58006005,107.410918,117.766501


#### 2.3.3. FAB model

**Note:** Remember to update the `periods` list below.

In [12]:
# Define time periods
periods = ['1983-1987', '1988-1992']

# Plot folder
png_fold = r'../../../processing_1983-1992/fab_pngs'

# Copy df
fab_df = df.copy()

# Loop over periods
for per in periods:
    print ('Processing %s...' % per)
    
    # Containers for data
    fab_ex_n = []
    fab_ex_s = []
    fab_ex_reg = []
    
    # Loop over rows
    for idx, row in df.iterrows():
        # Calc exceedance
        ex_n, ex_s, reg_id = cl.exceed_ns_icpm(row['clminn'], row['clmaxnoaa'],
                                               row['clmins'], row['clmaxsoaa'],
                                               row['N_%s' % per], row['S_%s' % per])
        
        # Add to result
        fab_ex_n.append(ex_n)
        fab_ex_s.append(ex_s)
        fab_ex_reg.append(reg_id)
        
        # Plot
#        out_png = os.path.join(png_fold, 'blr_%s_%s.png' % (idx, per))
#        cl.plot_critical_loads_func(row['clminn'], row['clmaxnoaa'],
#                                    row['clmins'], row['clmaxsoaa'],
#                                    [row['N_%s' % per],], [row['S_%s' % per]],
#                                    title='%s (%s)' % (idx, per), save_png=True, 
#                                    png_path=out_png)
#        plt.close()
        
    # Add to df
    fab_df['fab_ex_n_%s' % per] = fab_ex_n
    fab_df['fab_ex_s_%s' % per] = fab_ex_s
    fab_df['fab_ex_reg_%s' % per] = fab_ex_reg
    
    # Calc total ex = ex_n + ex_s
    fab_df['fab_ex_%s' % per] = fab_df['fab_ex_n_%s' % per] + fab_df['fab_ex_s_%s' % per]

# Get cols of interest
cols = [i for i in fab_df.columns if i[:3]=='fab']
fab_df = fab_df[cols]

# Set values below 0 to 0
fab_df[fab_df<0] = 0

# Write to output
out_path = r'../../../water/fab_exceedance_83-92.csv'
fab_df.to_csv(out_path)

print ('Finished.')

fab_df.head()

Processing 1983-1987...
Processing 1988-1992...
Finished.


,fab_ex_n_1983-1987,fab_ex_s_1983-1987,fab_ex_reg_1983-1987,fab_ex_1983-1987,fab_ex_n_1988-1992,fab_ex_s_1988-1992,fab_ex_reg_1988-1992,fab_ex_1988-1992
blr,,,,,,,,
58006001,82.414136,75.857767,2,158.271903,91.199850,80.286962,2,171.486812
58006002,85.098160,81.597006,2,166.695166,122.526732,101.122895,2,223.649626
58006003,98.078588,82.470368,2,180.548957,125.292874,94.759825,2,220.052699
58006004,74.303077,82.158453,2,156.461530,95.874506,88.521522,2,184.396028
58006005,98.513728,80.286962,2,178.800690,118.870871,90.642545,2,209.513416


#### 2.3.4. Area exceeded

In [13]:
# Read BLR grid
in_shp = r'../../../gis/shapefiles/blrgrid_uten_grums.shp'
blr_df = gpd.read_file(in_shp)

# Convert to 'normal' df
del blr_df['geometry']
blr_df = pd.DataFrame(blr_df)

# Get total area of Norway by summing cells
nor_area = blr_df['area_m2'].sum()/1E6

blr_df.head()

,BLR,area_m2
0,58005004,5.410322e+07
1,58006001,1.939285e+08
2,58006002,2.044114e+08
3,58006003,2.043394e+08
4,58006004,2.042695e+08


##### 2.3.4.1. SSWC

In [14]:
# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in sswc_df.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = sswc_df[sswc_df[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
sswc_areas = pd.DataFrame(data_dict)
sswc_areas.index = sswc_areas['period']
del sswc_areas['period']

# Pct exceed
sswc_areas['pct_exceed'] = 100*sswc_areas['area_exceeded_km2']/nor_area

# Tidy
sswc_areas = sswc_areas.round(0).astype(int)

# Write output
out_csv = r'../../../water/sswc_area_exceeded_83-92.csv'
sswc_areas.to_csv(out_csv)

sswc_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
1983-1987,99969,322183,31
1988-1992,81689,322183,25


##### 2.3.4.2. FAB

**Note:** Remember to update the `periods` list below.

In [15]:
# Get cols for overall exceedance
periods = ['1983-1987', '1988-1992']
cols = ['fab_ex_%s' % i for i in periods]
fab = fab_df[cols]

# Container for output
data_dict = {'period':[],
             'area_exceeded_km2':[],
             'area_total_km2':[]}

# Caclulate area exceeded
for col in fab.columns:
    per = col[-9:]
    
    # Get BLRs exceeded
    ex_blr = fab[fab[col]>0].index
    
    # Get area exceeded
    area_ex = blr_df['area_m2'][blr_df['BLR'].isin(ex_blr)].sum()/1E6
    
    # Add to output
    data_dict['period'].append(per)
    data_dict['area_exceeded_km2'].append(area_ex)
    data_dict['area_total_km2'].append(nor_area)
    
# Build df
fab_areas = pd.DataFrame(data_dict)
fab_areas.index = fab_areas['period']
del fab_areas['period']

# Pct exceed
fab_areas['pct_exceed'] = 100*fab_areas['area_exceeded_km2']/nor_area

# Tidy
fab_areas = fab_areas.round(0).astype(int)

# Write output
out_csv = r'../../../water/fab_area_exceeded_83-92.csv'
fab_areas.to_csv(out_csv)

fab_areas

,area_exceeded_km2,area_total_km2,pct_exceed
period,,,
1983-1987,139086,322183,43
1988-1992,116129,322183,36
